In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
import time
import pickle
import datetime

In [2]:
STATIC_FIELDS = ['SUBJECT_ID']
FIELDS = ['SUBJECT_ID', 'TimeStamp', 'Albumin', 'Alk. Phosphate', 'ALT', 'AST', 'Total Bili', 'BUN', 'Cholesterol',
          'Creatinine', 'Arterial BP [Diastolic]', 'FiO2 Set', 'GCS Total', 'Glucose', 'HCO3', 'Hematocrit',
          'Heart Rate', 'Potassium', 'Lactic Acid', 'Arterial BP Mean', 'MechVent', 'Magnesium', 'Sodium', 'NIDiasABP',
          'NIMAP', 'NISysABP', 'Arterial PaCO2', 'Arterial PaO2', 'Arterial pH', 'Platelets', 'Respiratory Rate',
          'SaO2', 'Arterial BP [Systolic]', 'Temperature C', 'TroponinI', 'TroponinT', 'Urine', 'WBC',
          'Previous WeightF']

In [3]:
df_all = pd.DataFrame(columns=FIELDS)
df_static_all = pd.DataFrame(columns=STATIC_FIELDS)

In [4]:
file_count = 0
unq_ids = []
time_i = time.time()

In [5]:
for filename in glob.glob('preprocess_data/24_hours/*.txt'):
    file_count += 1
    files = open(filename)
    #print(filename)
    file_list = []
    file_list_df = pd.read_csv(filename,sep=",")
    file_list = file_list_df.as_matrix()
    #display(file_list)
    unq_times = list(set(list(file_list_df['Time'])) - set(['Time']))
    #display(unq_times)
    unq_times = sorted(unq_times)
    id = file_list_df.loc[file_list_df.Parameter=='SUBJECT_ID', ['Value']].values[0][0]
    unq_ids.append(int(id))
    a = np.empty((len(unq_times),len(FIELDS),))
    a[:] = np.NAN
    a[:,0] = id
    df = pd.DataFrame(a,columns=FIELDS)
    df['TimeStamp'] = unq_times
    for i in range(len(file_list)):
        try:
            timestamp = file_list[i][0]
            var = file_list[i][1]
            val = file_list[i][2]
            df.loc[(df.TimeStamp == timestamp), [var]] = val
        except:
            continue
    df_static = pd.DataFrame(columns=STATIC_FIELDS)
    data = pd.DataFrame([[int(id)]], columns=STATIC_FIELDS)
    df_static =  df_static.append(data)
    df_static_all = df_static_all.append(df_static)
    df_all = df_all.append(df)
    if file_count%1000==0:
        print("done 1000")

/home/yoda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


done 1000
done 1000
done 1000
done 1000
done 1000
done 1000


In [6]:
df_all.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,02:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25,NaN,172,NaN,NaN,NaN,NaN,NaN,NaN
3,808,02:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22,NaN,157,NaN,NaN,NaN,NaN,NaN,56.299999237060547
4,808,02:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_static_all.head()

,SUBJECT_ID
0,808
0,5406
0,7107
0,2561
0,580


In [8]:
unq_ids[0]

808

In [9]:
len(unq_ids)

6587

In [10]:
a = [df_all,  df_static_all]
a

[    SUBJECT_ID TimeStamp Albumin Alk. Phosphate  ALT  AST Total Bili  BUN  \
 0          808     00:00     NaN            NaN  NaN  NaN        NaN  NaN   
 1          808     02:04     NaN            NaN  NaN  NaN        NaN  NaN   
 2          808     02:09     NaN            NaN  NaN  NaN        NaN  NaN   
 3          808     02:19     NaN            NaN  NaN  NaN        NaN  NaN   
 4          808     02:23     NaN            NaN  NaN  NaN        NaN  NaN   
 5          808     02:35     NaN            NaN  NaN  NaN        NaN  NaN   
 6          808     02:45     NaN            NaN  NaN  NaN        NaN  NaN   
 7          808     02:47     NaN            NaN  NaN  NaN        NaN  NaN   
 8          808     02:49     NaN            NaN  NaN  NaN        NaN  NaN   
 9          808     02:53     NaN            NaN  NaN  NaN        NaN  NaN   
 10         808     03:14     NaN            NaN  NaN  NaN        NaN  NaN   
 11         808     03:48     NaN            NaN  NaN  NaN      

In [11]:
with open('finalset/24hour_file.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [12]:
with open('finalset/24hour_file.pickle', 'rb') as handle:
    [df_all,  df_static_all] = pickle.load(handle)
    print("yes")
print("here")

yes
here


In [13]:
display(df_all.head())
df_static_all.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,02:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25,NaN,172,NaN,NaN,NaN,NaN,NaN,NaN
3,808,02:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22,NaN,157,NaN,NaN,NaN,NaN,NaN,56.299999237060547
4,808,02:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,SUBJECT_ID
0,808
0,5406
0,7107
0,2561
0,580


In [14]:
df_all['TimeStamp'] = df_all['TimeStamp'] + ':00'

In [15]:
df_all.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,02:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25,NaN,172,NaN,NaN,NaN,NaN,NaN,NaN
3,808,02:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22,NaN,157,NaN,NaN,NaN,NaN,NaN,56.299999237060547
4,808,02:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_all['TimeStamp'] = pd.to_timedelta(df_all['TimeStamp'])
df_all = df_all.reset_index(drop=True)
df_all.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,02:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25,NaN,172,NaN,NaN,NaN,NaN,NaN,NaN
3,808,02:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22,NaN,157,NaN,NaN,NaN,NaN,NaN,56.299999237060547
4,808,02:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
time_i = time.time()
i = 0
for id_ in unq_ids:
    if i%1000 == 0:
        print (i, time_i - time.time())
    i = i+1
    id1 = np.empty((1,39))
    id1 = id1.astype(str)
    id1[:] = np.NAN
    id1[0][0] = id_
    id1[0][1] = '24:59:59'
    df_add = pd.DataFrame(id1, columns=df_all.columns)
    df_all = df_all.append(df_add)
df_all.tail()    

0 -0.00010919570922851562
1000 -417.2401511669159
2000 -837.6539161205292
3000 -1256.7779750823975
4000 -1683.2478251457214
5000 -2112.7270033359528
6000 -2579.8869774341583


,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,13622,24:59:59,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,59314,24:59:59,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,29530,24:59:59,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,6973,24:59:59,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,27603,24:59:59,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [18]:
df_all['TimeStamp'] = pd.to_timedelta(df_all['TimeStamp'])
df_all = df_all.reset_index(drop=True)
df_all.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,02:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25,NaN,172,NaN,NaN,NaN,NaN,NaN,NaN
3,808,02:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22,NaN,157,NaN,NaN,NaN,NaN,NaN,56.299999237060547
4,808,02:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
type(df_all['Respiratory Rate'].values[1])

float

In [20]:
df_all['SUBJECT_ID'] = df_all['SUBJECT_ID'].astype('int')
df_all.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,02:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25,NaN,172,NaN,NaN,NaN,NaN,NaN,NaN
3,808,02:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22,NaN,157,NaN,NaN,NaN,NaN,NaN,56.299999237060547
4,808,02:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
len(df_all)

490678

In [22]:
for col in df_all.columns[2:]:
    for i in range(0,len(df_all[col].values)):
        if type(df_all[col].values[i])==type("hgfg"):
            try:
                df_all[col].values[i] = float(df_all[col].values[i])
            except:
                df_all[col].values[i] = np.nan
type(df_all['Respiratory Rate'].values[1])

float

In [23]:
df_all.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,02:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25,NaN,172,NaN,NaN,NaN,NaN,NaN,NaN
3,808,02:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22,NaN,157,NaN,NaN,NaN,NaN,NaN,56.3
4,808,02:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_all[df_all.columns[2:]] = df_all[df_all.columns[2:]].astype('float')
df_all.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,02:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25.0,NaN,172.0,NaN,NaN,NaN,NaN,NaN,NaN
3,808,02:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22.0,NaN,157.0,NaN,NaN,NaN,NaN,NaN,56.299999
4,808,02:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
res_24hour_df = pd.DataFrame(columns=df_all.columns)

In [26]:
res_24hour_df.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF


In [27]:
time_i = time.time()
i = 0
for id_ in unq_ids:
    if i%1000 == 0:
        print (i, time_i - time.time())
    i = i+1
    feat_id = df_all.loc[df_all['SUBJECT_ID'] == int(id_)].resample('1H', on='TimeStamp').agg(np.nanmean)
    feat_id['SUBJECT_ID'] = int(id_) 
    res_24hour_df = res_24hour_df.append(feat_id)
res_24hour_df.head()

0 -0.00016307830810546875


/home/yoda/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yoda/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/yoda/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/groupby.py:1062: RuntimeWarning: Mean of empty slice
  f = lambda x: func(x, *args, **kwargs)


1000 -62.13107132911682
2000 -125.94030785560608
3000 -190.45458841323853
4000 -257.2798252105713
5000 -330.8654794692993
6000 -411.181604385376


,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
00:00:00,808,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01:00:00,808,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02:00:00,808,02:31:33.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,21.75,NaN,165.0,NaN,NaN,NaN,NaN,NaN,56.299999
03:00:00,808,03:31:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.299999
04:00:00,808,04:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,19.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.299999


In [28]:
res_24hour_df['TimeStamp'] = res_24hour_df.index
res_24hour_df = res_24hour_df.reset_index(drop=True)
res_24hour_df.head()

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,808,01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,808,02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,21.75,NaN,165.0,NaN,NaN,NaN,NaN,NaN,56.299999
3,808,03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.299999
4,808,04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,19.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.299999


In [29]:
res_24hour_df.loc[res_24hour_df.SUBJECT_ID==824]

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
153850,824,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153851,824,0 days 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153852,824,0 days 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153853,824,0 days 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153854,824,0 days 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153855,824,0 days 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153856,824,0 days 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153857,824,0 days 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,116.0,NaN,...,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153858,824,0 days 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153859,824,0 days 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
res_24hour_df.shape

(164675, 39)

In [31]:
with open('finalset/res_24hour_df.pickle', 'wb') as handle:
    pickle.dump(res_24hour_df, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [32]:
res_24hour_df_backfill = pd.DataFrame(columns=res_24hour_df.columns)
time_i = time.time()
i = 0
for id_ in unq_ids:
    if i%1000 == 0:
        print (i, time_i - time.time())
    i = i+1
    feat_id_backfill = res_24hour_df.loc[res_24hour_df['SUBJECT_ID'] == int(id_)].fillna(method='bfill')
    #feat_id_backfill = feat_df.loc[df_all['ICUSTAY_ID'] == int(id_)].fillna(method='ffill')
    res_24hour_df_backfill = res_24hour_df_backfill.append(feat_id_backfill)
res_24hour_df_backfill.head()

0 -0.00014138221740722656
1000 -12.687005043029785
2000 -28.11013388633728
3000 -43.90090489387512
4000 -62.27265453338623
5000 -85.68154692649841
6000 -116.46397686004639


,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,21.75,NaN,165.0,NaN,NaN,NaN,NaN,6.1,56.299999
1,808,01:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,21.75,NaN,165.0,NaN,NaN,NaN,NaN,6.1,56.299999
2,808,02:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,21.75,NaN,165.0,NaN,NaN,NaN,NaN,6.1,56.299999
3,808,03:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,20.00,NaN,NaN,NaN,NaN,NaN,NaN,6.1,56.299999
4,808,04:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,19.00,NaN,NaN,NaN,NaN,NaN,NaN,6.1,56.299999


In [33]:
res_24hour_df_ffill = pd.DataFrame(columns=res_24hour_df.columns)
time_i = time.time()
i = 0
for id_ in unq_ids:
    if i%1000 == 0:
        print (i, time_i - time.time())
    i = i+1
    #feat_id_backfill = feat_df.loc[df_all['ICUSTAY_ID'] == int(id_)].fillna(method='bfill')
    feat_id_ffill = res_24hour_df_backfill.loc[res_24hour_df_backfill['SUBJECT_ID'] == int(id_)].fillna(method='ffill')
    res_24hour_df_ffill = res_24hour_df_ffill.append(feat_id_ffill)
res_24hour_df_ffill.head()

0 -0.0001571178436279297
1000 -10.730600595474243
2000 -23.082793712615967
3000 -37.074108600616455
4000 -52.29153847694397
5000 -73.65589380264282
6000 -103.02654528617859


,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,00:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,21.75,NaN,165.0,NaN,NaN,NaN,NaN,6.1,56.299999
1,808,01:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,21.75,NaN,165.0,NaN,NaN,NaN,NaN,6.1,56.299999
2,808,02:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,21.75,NaN,165.0,NaN,NaN,NaN,NaN,6.1,56.299999
3,808,03:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,20.00,NaN,165.0,NaN,NaN,NaN,NaN,6.1,56.299999
4,808,04:00:00,4.0,132.0,20.0,18.0,NaN,8.0,NaN,0.7,...,208.0,19.00,NaN,165.0,NaN,NaN,NaN,NaN,6.1,56.299999


In [34]:
normal_val = [-1]*39

In [36]:
i = 2
for col in res_24hour_df_ffill.columns[2:]:
    #print i
    res_24hour_df_ffill[col] = res_24hour_df_ffill[col].fillna(normal_val[i])
    i = i+1
res_24hour_df_ffill.head(50)

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,0 days 00:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,21.75,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
1,808,0 days 01:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,21.75,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
2,808,0 days 02:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,21.75,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
3,808,0 days 03:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,20.00,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
4,808,0 days 04:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,19.00,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
5,808,0 days 05:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,21.00,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
6,808,0 days 06:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,21.00,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
7,808,0 days 07:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,14.00,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
8,808,0 days 08:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,14.00,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999
9,808,0 days 09:00:00,4.0,132.0,20.0,18.0,-1.0,8.0,-1.0,0.7,...,208.0,21.00,-1.0,165.0,-1.0,-1.0,-1.0,-1.0,6.1,56.299999


In [37]:
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(res_24hour_df_ffill[res_24hour_df_ffill.columns[2:]])
res_24hour_df_ffill[res_24hour_df_ffill.columns[2:]] = X
res_24hour_df_ffill.head(50)

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,808,0 days 00:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.225248,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
1,808,0 days 01:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.225248,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
2,808,0 days 02:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.225248,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
3,808,0 days 03:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.207921,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
4,808,0 days 04:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.198020,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
5,808,0 days 05:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.217822,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
6,808,0 days 06:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.217822,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
7,808,0 days 07:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.148515,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
8,808,0 days 08:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.148515,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907
9,808,0 days 09:00:00,0.806452,0.032384,0.002796,0.001847,0.0,0.045455,0.0,0.012687,...,0.155045,0.217822,0.0,0.62406,0.0,0.0,0.0,0.0,0.013422,0.196907


In [38]:
res_24hour_df_ffill1 = res_24hour_df_ffill.sort_values(['SUBJECT_ID','TimeStamp'], ascending=[True,True])
res_24hour_df_ffill1 = res_24hour_df_ffill1.reset_index(drop=True)
res_24hour_df_ffill1.head(50)

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,17,0 days 00:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.060606,0.0,0.013433,...,0.103116,0.143564,0.990099,0.445489,0.0,0.0,0.0,0.0,0.021739,0.000000
1,17,0 days 01:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.060606,0.0,0.013433,...,0.177300,0.143564,0.990099,0.445489,0.0,0.0,0.0,0.0,0.047259,0.000000
2,17,0 days 02:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.060606,0.0,0.013433,...,0.177300,0.128713,0.990099,0.458647,0.0,0.0,0.0,0.0,0.047259,0.000000
3,17,0 days 03:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.188119,0.990099,0.379699,0.0,0.0,0.0,0.0,0.047259,0.000000
4,17,0 days 04:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.237624,0.990099,0.439850,0.0,0.0,0.0,0.0,0.047259,0.000000
5,17,0 days 05:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.217822,0.990099,0.439850,0.0,0.0,0.0,0.0,0.047259,0.000000
6,17,0 days 06:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.217822,0.990099,0.439850,0.0,0.0,0.0,0.0,0.047259,0.000000
7,17,0 days 07:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.168317,0.990099,0.428571,0.0,0.0,0.0,0.0,0.047259,0.000000
8,17,0 days 08:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.168317,0.990099,0.375940,0.0,0.0,0.0,0.0,0.047259,0.000000
9,17,0 days 09:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.198020,0.990099,0.402256,0.0,0.0,0.0,0.0,0.047259,0.000000


In [39]:
matrix3D = np.array(res_24hour_df_ffill1.drop(['SUBJECT_ID', 'TimeStamp'], 1))
matrix3D

array([[0.        , 0.        , 0.        , ..., 0.        , 0.02173913,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.04725898,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.04725898,
        0.        ],
       ...,
       [0.79032258, 0.        , 0.00266276, ..., 0.        , 0.01587902,
        0.        ],
       [0.79032258, 0.        , 0.00266276, ..., 0.        , 0.01587902,
        0.        ],
       [0.79032258, 0.        , 0.00266276, ..., 0.        , 0.01587902,
        0.        ]])

In [40]:
matrix3D.shape

(164675, 37)

In [41]:
p = []
ff = res_24hour_df_ffill1.groupby('SUBJECT_ID')
for s,g in ff:
    p.append((s,len(g)))

In [42]:
for k in p:
    if k[1]!=25:
        print(k[0],k[1])

In [43]:
#gg = res_24hour_df_ffill1[res_24hour_df_ffill1['SUBJECT_ID']==16351].drop_duplicates(subset=['TimeStamp'])
#gg

In [44]:
#res_24hour_df_ffill1 = res_24hour_df_ffill1[res_24hour_df_ffill1['SUBJECT_ID']!=16351]
#res_24hour_df_ffill1

In [45]:
#res_24hour_df_ffill1 = res_24hour_df_ffill1.append(gg)

In [47]:
res_24hour_df_ffill1.head(100)

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,17,0 days 00:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.060606,0.0,0.013433,...,0.103116,0.143564,0.990099,0.445489,0.000000,0.0,0.0,0.0,0.021739,0.000000
1,17,0 days 01:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.060606,0.0,0.013433,...,0.177300,0.143564,0.990099,0.445489,0.000000,0.0,0.0,0.0,0.047259,0.000000
2,17,0 days 02:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.060606,0.0,0.013433,...,0.177300,0.128713,0.990099,0.458647,0.000000,0.0,0.0,0.0,0.047259,0.000000
3,17,0 days 03:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.188119,0.990099,0.379699,0.000000,0.0,0.0,0.0,0.047259,0.000000
4,17,0 days 04:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.237624,0.990099,0.439850,0.000000,0.0,0.0,0.0,0.047259,0.000000
5,17,0 days 05:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.217822,0.990099,0.439850,0.000000,0.0,0.0,0.0,0.047259,0.000000
6,17,0 days 06:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.217822,0.990099,0.439850,0.000000,0.0,0.0,0.0,0.047259,0.000000
7,17,0 days 07:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.168317,0.990099,0.428571,0.000000,0.0,0.0,0.0,0.047259,0.000000
8,17,0 days 08:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.168317,0.990099,0.375940,0.000000,0.0,0.0,0.0,0.047259,0.000000
9,17,0 days 09:00:00,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.198020,0.990099,0.402256,0.000000,0.0,0.0,0.0,0.047259,0.000000


In [48]:
res_24hour_df_ffill1 = res_24hour_df_ffill1.sort_values(by='SUBJECT_ID')

In [49]:
len(res_24hour_df_ffill1['SUBJECT_ID'].unique())

6587

In [50]:
matrix3D = np.array(res_24hour_df_ffill1.drop(['SUBJECT_ID', 'TimeStamp'], 1))

In [51]:
matrix3D.shape

(164675, 37)

In [52]:
matrix3D = np.array(matrix3D).reshape((6587, 25, 37))
matrix3D

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.02173913, 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.04725898, 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.04725898, 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.04725898, 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.04725898, 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.04725898, 0.        ]],

       [[0.        , 0.01777453, 0.01650912, ..., 0.        ,
         0.04177694, 0.22886597],
        [0.        , 0.01777453, 0.01650912, ..., 0.        ,
         0.04177694, 0.22886597],
        [0.        , 0.01777453, 0.01650912, ..., 0.        ,
         0.04177694, 0.22886597],
        ...,
        [0.        , 0.01777453, 0.01650912, ..., 0.        ,
         0.04177694, 0.22886597],
        [0. 

In [53]:
matrix3D.shape

(6587, 25, 37)

In [54]:
outcomes = pd.read_csv('outcome.csv')
outcomes.head()

,SUBJECT_ID,LABEL
0,17,0
1,21,0
2,23,0
3,34,0
4,36,1


In [55]:
outcomes = outcomes.sort_values(by='SUBJECT_ID')
outcomes

,SUBJECT_ID,LABEL
0,17,0
1,21,0
2,23,0
3,34,0
4,36,1
5,61,0
6,68,1
7,85,0
8,94,0
9,103,1


In [56]:
patient_data = (res_24hour_df_ffill1.SUBJECT_ID.unique())
outcome_data =  (outcomes.SUBJECT_ID.unique())
for i in range(0,len(outcome_data)):
    if patient_data[i]!=outcome_data[i]:
        print("problem")

In [57]:
with open('finalset/preprocess_24_hours.pickle', 'wb') as handle:
    pickle.dump([matrix3D, outcomes], handle, protocol = pickle.HIGHEST_PROTOCOL)

In [58]:
outcomes.to_csv("finalset/outcomes.csv", index=False)

In [59]:
res_24hour_df_ffill1.to_csv("finalset/patient_data_24_hours.csv", index=False)